# OpenAI Fine-Tuning

Once you've captured run traces from your deployment (production or beta), it's likely you'll want to use that data to
fine-tune a model. This walkthrough will show a quick way to do so.

Steps:

1. List LLM runs (optionally filtering by project, time, tags, etc.)
   - [Optional] Create a 'training' dataset to keep track of the data used for this model.
3. Convert runs to OpenAI messages or another ormat)
5. Fine-tune and use new model.

In [1]:
from langsmith import Client

client = Client()

## 1. Get the LLM runs

For examples of more 'advanced' filtering, check out the filtering[guide](https://docs.smith.langchain.com/tracing/use-cases/export-runs/local)

In [2]:
import datetime

project_name = "chat-langchain"
run_type = "llm"
end_time = datetime.datetime.now()

runs = client.list_runs(
        project_name=project_name,
        run_type=run_type,
        end_time=end_time,
        error=False,
)

## 1a: (Recommended) Add to a training dataset

While not necessary for the fast-path of making your first fine-tuned model, datasets help build in a principled way by helping track the exact data used in a given model.

In [31]:
dataset = client.create_dataset(
    dataset_name = "Fine-Tuning Dataset Example", 
    description=f"Chat logs taken from project {project_name} for fine-tuning",
)
for run in runs:
    if 'messages' not in run.inputs or not run.outputs:
        # Filter out non chat runs
        continue
    try:
        client.create_example(
            dataset_id=dataset.id,
            inputs=run.inputs, 
            outputs=run.outputs,
        )
    except:
        # Duplicates
        pass

## 2. Load examples as messages

We will first load the messages as LangChain objects then take advantage of the OpenAI adapter helper to convert these
to dictionaries in the form expected by OpenAI's fine-tuning endpoint.

In [6]:
from langsmith import schemas
from langchain.load import load

def convert_messages(example: schemas.Example) -> dict:
    messages = load.load(example.inputs)['messages']
    message_chunk = load.load(example.outputs)['generations'][0]['message']
    return {"messages": messages + [message_chunk]}

In [7]:
messages = [
    convert_messages(example)
    for example in client.list_examples(dataset_name="Fine-Tuning Dataset Example")
]

Now that we have the traces back as LangChain message objects, you can use the adapters
to convert to other formats, such as OpenAI's fine-tuning format.

In [8]:
from langchain.adapters import openai as openai_adapter

finetuning_messages = openai_adapter.convert_messages_for_finetuning(messages)

## 3. Finetune

Now you can use these message dictionaries for downstream tasks like fine-tuning. Note that the OpenAI API doesn't currently support the 'function_call' argument when fine-tuning. We will filter these out first here. It may be that this requirement is relaxed by the time you read this guide.

In [13]:
import time
import json
import io

import openai

my_file = io.BytesIO()
for group in finetuning_messages:
    if any(["function_call" in message for message in group]):
        continue
    my_file.write((json.dumps({"messages": group}) + "\n").encode('utf-8'))

my_file.seek(0)
training_file = openai.File.create(
  file=my_file,
  purpose='fine-tune'
)

# Wait while the file is processed
status = openai.File.retrieve(training_file.id).status
start_time = time.time()
while status != "processed":
    print(f"Status=[{status}]... {time.time() - start_time:.2f}s", end="\r", flush=True)
    time.sleep(5)
    status = openai.File.retrieve(training_file.id).status
print(f"File {training_file.id} ready after {time.time() - start_time:.2f} seconds.")

File file-ixTtVCKDGrZ7PiVZFszQr6kN ready after 30.55 seconds.


Next, fine-tune the model. This could take 10+ minutes.

In [14]:
job = openai.FineTuningJob.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo",
)

# It may take 10-20+ minutes to complete training.
status = openai.FineTuningJob.retrieve(job.id).status
start_time = time.time()
while status != "succeeded":
    print(f"Status=[{status}]... {time.time() - start_time:.2f}s", end="\r", flush=True)
    time.sleep(5)
    job = openai.FineTuningJob.retrieve(job.id)
    status = job.status

In [34]:
from langchain import chat_models, prompts

model_name = job.fine_tuned_model
# Example: ft:gpt-3.5-turbo-0613:personal::5mty86jblapsed
model = chat_models.ChatOpenAI(model=model_name)
chain.invoke({"input": "Who are you designed to assist?"})

AIMessage(content='I am designed to assist anyone who has questions or needs help related to LangChain. This could include developers, users, or anyone else who is interested in or using LangChain.', additional_kwargs={}, example=False)

## Conclusion

Congratulations! You've fine-tuned a model on your traced LLM runs.

This is an extremely simple recipe that demonstrates the end-to-end workflow.
It is likely that you will want to use various methods to filter, fix, and observe the data you choose to fine-tune the model on. We welcome additional recipes of things that have worked for you!